In [2]:
import fc_trainer
import alg_gen
import subprocess
import random

import importlib

importlib.reload(fc_trainer)

from fc_trainer import FirstCenter
fc = FirstCenter()

from alg_gen import AlgGen
ag = AlgGen()

In [4]:
scramble = "B' D B' R D' B R D U' B U' B R' B L U B' D' L R' BR R L' U' BL B BL"
solutions = fc.generate_solutions_twizzle(scramble, count=5)

sorted(solutions, key= lambda x : x[2])[:20]

[('{F, BR}', "L D BR' D' R", 5),
 ('{F, BR}', "L D' BL' D R", 5),
 ('{F, BR}', "L' D BL L' R", 5),
 ('{U, F}', "L BL' D' B' R' L", 6),
 ('{U, BR}', "F B' D BL' L R", 6),
 ('{F, BR}', "F L F' R' D R'", 6),
 ('{F, BR}', "F R' F' L D R", 6),
 ('{F,U}', "B D F' D' L U'", 6),
 ('{F,U}', "B D' BR' D L U'", 6),
 ('{F,U}', "B L U' L D L'", 6),
 ('{F,U}', "B' D BR B' L U'", 6),
 ('{F, BL}', "R B U L D' L'", 6),
 ('{F, BL}', "R D BR' B U R'", 6),
 ('{F, BL}', "R D BL' D' B U", 6),
 ('{F, BL}', "R D' F' D B U", 6),
 ('{F, BL}', "R U BR' R D' R", 6),
 ('{BR, U}', "BR B' L R' U' R", 6),
 ('{BR, U}', "BR' D BL D' L B", 6),
 ('{BR, U}', "BR' BL D' B' L B'", 6),
 ('{BR, BL}', "BL L' R B' BR' R'", 6)]

In [52]:
import csv

solutions = []

with open("FTO_FC.csv", 'r') as f:
    for line in f.readlines():
        solutions.append((line.split(',')[0], int(line.split(',')[1][1])))


In [122]:
movecount = 4
case = random.sample(list(filter(lambda x: x[1] == movecount, solutions)), 1)[0]

with open("./twizzle/first_center.scr", 'w') as f:
        f.write("ScrambleAlg FC\n")
        f.write(f"{case[0]}\n")
        f.write("End\n")

twsearch_path = "/home/aedan/Desktop/cubing/twsearch/build/bin/twsearch"

p = subprocess.Popen(f"{twsearch_path} -M 8096 -c 1 --mindepth 10 --quiet --randomstart twizzle/FTO_FC.tws twizzle/first_center.scr", shell = True, stdout=subprocess.PIPE)

pOut = p.communicate()
p.kill()

pOut = pOut[0].decode().split("\n")[1:-2]
pOut = [x.strip() for x in pOut if "found" not in x.lower()]

print(invert(pOut[0]))

U L' R' BL BR D' R' U' B BR


In [118]:
invert(case[0])

"BL B BR R'"

In [90]:
# unwiden algs written with BRw and Dw moves
def unwiden(case):
    start_centers = ["Dw", "D", "R", "BL", "BR", "BRw", "F", "B"]
    current_centers = ["U", "D", "R", "BL", "BR", "L", "F", "B"]

    new_case = ""
    case.strip()

    for move in case.split(" "):
        suffix = ""
        if "'" in move:
            move = move.removesuffix("'")
            suffix = "'"
        
        if move == "BRw"  and suffix != "'":
            current_centers[0], current_centers[3], current_centers[6] = current_centers[6], current_centers[0], current_centers[3]
            current_centers[2], current_centers[7], current_centers[1] = current_centers[1], current_centers[2], current_centers[7]
            new_case += " " + current_centers[start_centers.index(move)] + suffix
        elif move == "Dw" and suffix == "'":
            current_centers[2], current_centers[5], current_centers[7] = current_centers[7], current_centers[2], current_centers[5]
            current_centers[3], current_centers[4], current_centers[6] = current_centers[6], current_centers[3], current_centers[4]
            new_case += " " + current_centers[start_centers.index(move)] + suffix
        elif move == "BRw" and suffix == "'":
            current_centers[0], current_centers[3], current_centers[6] = current_centers[3], current_centers[6], current_centers[0]
            current_centers[2], current_centers[7], current_centers[1] = current_centers[7], current_centers[1], current_centers[2]
            new_case += " " + current_centers[start_centers.index(move)] + suffix
        elif move == "Dw" and suffix != "'":
            current_centers[2], current_centers[5], current_centers[7] = current_centers[5], current_centers[7], current_centers[2]
            current_centers[3], current_centers[4], current_centers[6] = current_centers[4], current_centers[6], current_centers[3]
            new_case += " " + current_centers[start_centers.index(move)] + suffix
        else:
            new_case += " " + current_centers[start_centers.index(move)] + suffix
    
    return new_case.strip()

def invert(case):
    inverted = []
    for move in case.split()[::-1]:
        if "'" in move:
            inverted.append(move.removesuffix("'"))
        else:
            inverted.append(move + "'")

    return ' '.join(inverted)

"R B' U' F' R'"